# Notebook to Cluster Neighborhoods in Toronto

#### Importing Libraries

In [3]:
import numpy as np
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup

#### Scraping Wikipedia Page

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [5]:
page = urllib.request.urlopen(url)

In [6]:
soup = BeautifulSoup(page, "lxml")

In [7]:
table=soup.find('table', class_='wikitable sortable')

#### Getting the required table and cleaning

In [8]:
postal_codes = []
boroughs = []
neighborhoods = []
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        postal_codes.append(cells[0].find(text=True))
        boroughs.append(cells[1].find(text=True))
        neighborhoods.append(cells[2].find(text=True))

In [9]:
for i in range(len(postal_codes)):
    postal_codes[i] = postal_codes[i].replace('\n','')
    boroughs[i] = boroughs[i].replace('\n','')
    neighborhoods[i] = neighborhoods[i].replace('\n','')

#### Converting to Dataframe

In [10]:
df = pd.DataFrame(postal_codes, columns=['PostalCode'])
df['Borough'] = boroughs
df['Neighborhoods'] = neighborhoods
df.head()

,PostalCode,Borough,Neighborhoods
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Removing unassigned Boroughs

In [11]:
new_df = df[df['Borough'] != 'Not assigned'].reset_index().drop('index',axis=1)

In [12]:
new_df.head()

,PostalCode,Borough,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [13]:
new_df.shape

(103, 3)

#### Checking to see if all rows have unique postal code

In [14]:
counts = new_df['PostalCode'].value_counts()
counts[counts > 1]

Series([], Name: PostalCode, dtype: int64)

Since we have an empty series, all the rows have unique postal code

In [15]:
new_df.shape

(103, 3)

#### Get Geospatial Data

In [16]:
geo_data = pd.read_csv('https://cocl.us/Geospatial_data')

In [17]:
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge Both dataframes into final df

In [23]:
new_df = new_df.merge(geo_data, left_on = 'PostalCode', right_on = 'Postal Code').drop('Postal Code', axis = 1)


In [26]:
new_df.head(20)

,PostalCode,Borough,Neighborhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [27]:
new_df.shape

(103, 5)